## Requirements

- Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design.
- 30% of the company have at least 1 child.
- Developers like to be near successful tech startups that have raised at least 1 Million dollars.
- Executives like Starbucks A LOT. Ensure there's a starbucks not to far.
- Account managers need to travel a lot
- All people in the company have between 25 and 40 years, give them some place to go to party.
- Nobody in the company likes to have companies with more than 10 years in a radius of 2 KM.
- The CEO is Vegan

In [106]:
from pymongo import MongoClient

Function to conect to the database and collection of MongoDB in localhost:

In [116]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

Function to get the latitude and longitude in geoJSON format:

In [117]:
db, coll = connectCollection('companies','companies')

In [122]:
def getLocation(comp,i):
    latitude = comp['offices'][i]['latitude']
    longitude = comp['offices'][i]['longitude']
    loc = {
        'type':'Point',
        'coordinates':[latitude, longitude]
    }
    return loc

Let's find all the companies that have less than 10 years since their foundation:

In [123]:
company = coll.find({'founded_year': {'$gt': 2009}})

Now I set a new field in the documents, call "location", which will contain the info provided by getLocation function:

In [111]:
getLocation(company[3])

{'type': 'Point', 'coordinates': [33.8171, -111.9035]}

In [125]:
for c in company:
    for i in range(len(c['offices'])):
        value = {"$set": {'location'+str(i):getLocation(c,i)}}
        coll.update_one(c,value)

        

In [120]:
for c in company:
    value = {"$unset": {'location':''}}
    coll.update_one(c,value)